In [1]:
from __future__ import print_function
"""
Tutorial: A reference implementation of non-hermitian cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

$\langle \mu \rangle$ represents the dipole expectation value computed at the Hartree-Fock level.
The dipole operator has the form (in first quantization):
$$ \hat{\mu} = \sum_{i}^{N_e} \mu(x_i) + \sum_{A}^{N_N} \mu_{nuc}(x_A) $$
where $\mu(x_i)$ depends on electronic coordinates and $\mu_{nuc}(x_A)$ depends on nuclear coordinates.

We will check to make sure we can compute the RHF dipole expectation
value using the dipole integrals below.

In [2]:
# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
import scipy.linalg as la

# ==> Set Basic Psi4 Options <==

# Memory specifications
psi4.set_memory(int(2e9))
numpy_memory = 2

# Output options
psi4.core.set_output_file('output.dat', False)

Define the molecule!

In [3]:
mol = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

We use Psi4 to compute the RHF energy and wavefunction and store them in variables `scf_e` and `scf_wfn`. We also check the memory requirements for computation:

In [13]:
# Get the SCF wavefunction & energies
scf_e, wfn = psi4.energy('scf', return_wfn=True)

# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()
nmo = wfn.nmo()

#E, wfn = energy('hf', return_wfn=True)
props = psi4.oeprop(wfn, 'DIPOLE', 'QUADRUPOLE')
print(props)

None


In [14]:
E, wfn = prop('scf', properties=["GRID_ESP", "GRID_FIELD"], return_wfn=True)
Vvals = wfn.oeprop.Vvals()
Exvals = wfn.oeprop.Exvals()
Eyvals = wfn.oeprop.Eyvals()
Ezvals = wfn.oeprop.Ezvals()

NameError: name 'prop' is not defined

We first obtain orbital information from our wavefunction. We also create an instance of MintsHelper to help build our molecular integrals:

In [5]:
# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(wfn.basisset())

# Grab data from wavfunction

# number of doubly occupied orbitals
ndocc   = wfn.nalpha()

# total number of orbitals
nmo     = wfn.nmo()

# number of virtual orbitals
nvirt   = nmo - ndocc

# orbital energies
eps     = np.asarray(wfn.epsilon_a())

# occupied orbitals:
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals:
Cv = wfn.Ca_subset("AO", "VIR")

# grab all transformation vectors and store to a numpy array!
C = np.asarray(wfn.Ca())

We can transform the dipole integrals from the AO to the MO basis in the following way:
\begin{equation}
{\bf \mu}^{\xi} = {\bf C}^T {\bf \mu}^{\xi}_{ao} {\bf C},
\end{equation}
where ${\bf \mu}^{\xi}_{ao}$ represents the $\xi$ component of the dipole integrals in the AO basis and ${\bf C}$
represents the matrix of transformation vectors that go from AOs to MOs.  Note here 
$\xi$ can be $x$, $y$, or $z$.  At first pass, we will consider only the $z$ component, which means that 
the molecule will only couple to photons polarized along the $z$ axis.

In [6]:
# ==> array of dipole integrals <==
# start by assuming photon polarized along z so only
# need z-component of molecular dipole integrals!
mu_ao_z = np.asarray(mints.ao_dipole()[2])

# transform to the MO basis
mu_z = np.dot(C.T, mu_ao_z).dot(C)

### if you want to see the arrays, uncomment here!
#print("z-component of the AO dipole matrix")
#print(mu_ao_z)
#print("z-component of the MO dipole matrix")
#print(mu_z)
#print(len(mu_z))



In [7]:
# Get the z-component of the RHF dipole moment from psi4.
# this will be in debye, and we will want to convert it into
# au to match those computed from dipole integrals
muz_psi4_debye = psi4.core.variable('SCF DIPOLE Z')
qz_psi4_debye = psi4.core.variable('SCF QUADRUPOLE Z')
muz_psi4_au = muz_psi4_debye/2.54174623

# Get the nuclear dipole moment from psi4
nuc_dipole = mol.nuclear_dipole()

# now try to compute the dipole expectation value from the 
# dipole integrals
ed_z = 0.
for i in range(0, ndocc):
    ed_z += mu_z[i,i]

muz_computed = 2 * ed_z + nuc_dipole[2]    
print("Computed from integrals:",muz_computed)
print("Psi4:                   ",muz_psi4_au)
print("These dipole moments are close:",np.isclose(muz_computed,muz_psi4_au))



<ipython-input-7-bb9be6127903>:4: FutureWarning: Using scalar QCVariable `SCF DIPOLE Z` [D] instead of array `SCF DIPOLE` [e a0] is deprecated, and in 1.5 it will stop working

  muz_psi4_debye = psi4.core.variable('SCF DIPOLE Z')


KeyError: 'psi4.core.variable: Requested variable SCF QUADRUPOLE Z was not set!\n'